## Setup

In [1]:
#pip install --upgrade git+https://github.com/keras-team/keras-cv -q

In [2]:
from tqdm.auto import tqdm
import tensorflow as tf
import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

/home/andrea/tfenvl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-02 09:44:36.715836: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-02 09:44:36.921253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 09:44:36.921293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 09:44:36.956659: E e

Using TensorFlow backend


In [3]:
class_ids = [
    #"TV", "bed", "chair", "clock", "console", "consoleeeeee", "door", "fan", "light", "sofa", "switchboard", "table"
    #"emptychair", "fullchair"
    #"Chair","Sofa","Table"
    "Cane", "Gatto"
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [4]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_m_backbone_coco",  # We will use yolov8 small backbone with coco weights
    load_weights=True
)
backbone.trainable = False

2024-03-02 09:44:42.757217: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 09:44:42.872786: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 09:44:42.872828: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 09:44:42.877681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 09:44:42.877771: I external/local_xla/xla/stream_executor

In [5]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    #fpn_depth=1
)

In [6]:
yolo.load_weights("model-m-bbfreez-animali.h5")
#yolo = keras_cv.models.load("model.h5")

In [9]:
yolo.prediction_decoder = keras_cv.layers.NonMaxSuppression(
    bounding_box_format="xyxy",
    from_logits=False,
    iou_threshold=0.1,
    confidence_threshold=0.05,
    max_detections=50
)

In [11]:
def visualize_generic(model, image_name, bounding_box_format):
    h = 640
    w = 640
    #images = load_image(os.path.join(path_images, image_name))
    orig_image = tf.io.read_file(image_name)
    orig_image = tf.image.decode_jpeg(orig_image, channels=3)
    orig_h = orig_image.shape[0]
    orig_w = orig_image.shape[1]
    rh = orig_h/h
    rw = orig_w/w

    image = tf.image.resize(orig_image, [h,w])
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, axis=0)
    
    orig_image = tf.cast(orig_image, tf.float32)
    orig_image = tf.expand_dims(orig_image, axis=0)

    y_pred = model.predict(image)
    #y_pred = bounding_box.to_ragged(y_pred)

    visualization.plot_bounding_box_gallery(
        image,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        #y_true={},
        y_pred=y_pred,
        scale=4,
        rows=1,
        cols=1,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )

    print(y_pred["boxes"][0][:2])
    boxes = y_pred["boxes"]
    for box in boxes[0]:
        box[0] = box[0]*rw
        box[1] = box[1]*rh
        box[2] = box[2]*rw
        box[3] = box[3]*rh
    print(y_pred["boxes"][0][:2])

    visualization.plot_bounding_box_gallery(
        orig_image,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        #y_true={},
        y_pred=y_pred,
        scale=4,
        rows=1,
        cols=1,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )
    
    # y_pred["boxes"] = tf.squeeze(y_pred["boxes"], axis=0)
    # visualization.draw_bounding_boxes(
    #     image,
    #     y_pred,
    #     color= "red",
    #     bounding_box_format=bounding_box_format,
    #     class_mapping=class_mapping
    # )

# visualize_generic(yolo, image_name="../dataset1/test/00012_jpg.rf.f54b7ef0968cf6b0f990928d169e1a83.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset1/test/00041_jpg.rf.1b685611d13e60bfd19b1ad008eb81c9.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset1/test/00059_jpg.rf.f69bbdb4d09580675715e9a203470631.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset3/test/Chair-269-_jpg.rf.3a81b0889599ce42bc735ff199f7e536.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset3/test/Sofa-359-_jpg.rf.2995061f9eaae4bca7913b736e3e49a1.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset3/test/Chairs1-16-_jpg.rf.a4506ce899294f71c7fd534a42534144.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_1.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_2.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_3.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_4.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_5.jpg", bounding_box_format="xyxy")

visualize_generic(yolo, image_name="../animali/valid/cane-e-gatto-insieme.jpg", bounding_box_format="xyxy")
visualize_generic(yolo, image_name="../animali/valid/IMG_0206-768x513.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../animali/valid/183005673-110464f6-fd3c-42b8-9c38-553f0e46a50d.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../animali/valid/cani-gatti-litigano-m.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../animali/valid/dog-2606759_640.jpg", bounding_box_format="xyxy")

#img_0120_jpg.rf.36da0f1b2f59ec13cb70e1b82defd9a0.jpg

1/1 [==============================] - 1s 608ms/step


KeyboardInterrupt: 